In [1]:
!nvidia-smi

Thu Jun 29 23:34:50 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [10]:
!mkdir data
!wget https://transfer.sh/3K5kSPhAmc/IMDB%20Dataset.csv -O ./data/IMDB_Dataset.csv

--2023-06-29 23:41:23--  https://transfer.sh/3K5kSPhAmc/IMDB%20Dataset.csv
Resolving transfer.sh (transfer.sh)... 144.76.136.153, 2a01:4f8:200:1097::2
Connecting to transfer.sh (transfer.sh)|144.76.136.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 66212309 (63M) [text/csv]
Saving to: ‘./data/IMDB_Dataset.csv’

./data/IMDB_Dataset 100%[===================>]  63.14M  14.3MB/s    in 5.8s    

2023-06-29 23:41:30 (10.8 MB/s) - ‘./data/IMDB_Dataset.csv’ saved [66212309/66212309]



In [11]:
import tensorflow as tf
import pandas as pd
import numpy as np

### Loading the data

In [12]:
data = pd.read_csv("./data/IMDB_Dataset.csv")

In [13]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


#### Splitting test & train

In [14]:
data = data.sample(frac=1, random_state=42)

In [15]:
train_proportion = 0.8

In [16]:
train_rows = int(train_proportion * len(data))
test_rows = len(data) - train_rows

In [17]:
train_data = data[:train_rows]
test_data = data[train_rows:]

In [18]:
train_data.head()

,review,sentiment
33553,I really liked this Summerslam due to the look...,positive
9427,Not many television shows appeal to quite as m...,positive
199,The film quickly gets to a major chase scene w...,negative
12447,Jane Austen would definitely approve of this o...,positive
39489,Expectations were somewhat high for me when I ...,negative


In [19]:
test_data.head()

,review,sentiment
7928,Warning: This review contains a spoiler.<br />...,negative
2107,In order to stop her homosexual friend Albert ...,positive
30363,"Wow this was a great Italian ""ZOMBIE"" movie by...",positive
31262,"Making a film based on a true story, particula...",negative
42135,"Bettie Page was a icon of the repressed 1950s,...",positive


In [20]:
train_data, train_labels = train_data["review"], train_data["sentiment"]
test_data, test_labels = test_data["review"], test_data["sentiment"]

In [21]:
train_data.head()

33553    I really liked this Summerslam due to the look...
9427     Not many television shows appeal to quite as m...
199      The film quickly gets to a major chase scene w...
12447    Jane Austen would definitely approve of this o...
39489    Expectations were somewhat high for me when I ...
Name: review, dtype: object

In [22]:
test_data.head()

7928     Warning: This review contains a spoiler.<br />...
2107     In order to stop her homosexual friend Albert ...
30363    Wow this was a great Italian "ZOMBIE" movie by...
31262    Making a film based on a true story, particula...
42135    Bettie Page was a icon of the repressed 1950s,...
Name: review, dtype: object

In [23]:
train_data = train_data.apply(lambda review: review.split())
test_data = test_data.apply(lambda review: review.split())

In [24]:
train_data.head()

33553    [I, really, liked, this, Summerslam, due, to, ...
9427     [Not, many, television, shows, appeal, to, qui...
199      [The, film, quickly, gets, to, a, major, chase...
12447    [Jane, Austen, would, definitely, approve, of,...
39489    [Expectations, were, somewhat, high, for, me, ...
Name: review, dtype: object

In [25]:
test_data.head()

7928     [Warning:, This, review, contains, a, spoiler....
2107     [In, order, to, stop, her, homosexual, friend,...
30363    [Wow, this, was, a, great, Italian, "ZOMBIE", ...
31262    [Making, a, film, based, on, a, true, story,, ...
42135    [Bettie, Page, was, a, icon, of, the, represse...
Name: review, dtype: object

### Creating word indexes

In [26]:
text_data = data["review"].tolist()

In [27]:
text_data[:5]

["I really liked this Summerslam due to the look of the arena, the curtains and just the look overall was interesting to me for some reason. Anyways, this could have been one of the best Summerslam's ever if the WWF didn't have Lex Luger in the main event against Yokozuna, now for it's time it was ok to have a huge fat man vs a strong man but I'm glad times have changed. It was a terrible main event just like every match Luger is in is terrible. Other matches on the card were Razor Ramon vs Ted Dibiase, Steiner Brothers vs Heavenly Bodies, Shawn Michaels vs Curt Hening, this was the event where Shawn named his big monster of a body guard Diesel, IRS vs 1-2-3 Kid, Bret Hart first takes on Doink then takes on Jerry Lawler and stuff with the Harts and Lawler was always very interesting, then Ludvig Borga destroyed Marty Jannetty, Undertaker took on Giant Gonzalez in another terrible match, The Smoking Gunns and Tatanka took on Bam Bam Bigelow and the Headshrinkers, and Yokozuna defended t

In [28]:
tokenizer = tf.keras.preprocessing.text.Tokenizer()

In [29]:
tokenizer.fit_on_texts(text_data)

In [30]:
word_indexes = tokenizer.word_index

In [31]:
len(word_indexes), list(word_indexes.items())[:5]

(124252, [('the', 1), ('and', 2), ('a', 3), ('of', 4), ('to', 5)])

#### Adjusting the word indexes

In [32]:
for word, index in word_indexes.items():
    word_indexes[word] = index + 3

In [33]:
list(word_indexes.items())[:5]

[('the', 4), ('and', 5), ('a', 6), ('of', 7), ('to', 8)]

In [34]:
word_indexes["<PAD>"] = 0
word_indexes["<UNK>"] = 1
word_indexes["<UNUSED>"] = 2
word_indexes["<br />"] = 3
word_indexes["I"] = len(word_indexes)
word_indexes["Summerslam"] = len(word_indexes)

In [35]:
word_indexes = dict(sorted(word_indexes.items(), key=lambda x: x[1]))

In [36]:
print("Num. of vocabulary:", len(list(word_indexes.items())))

Num. of vocabulary: 124258


#### Encoders

In [37]:
def review_encoder(review: str):
    result = []
    punctuation_list = [",", ".", "!", "?", "...", ";", ":"]
    for word in review:
        if word[-1] in punctuation_list:
            word = word[:len(word)-1]
        if word in word_indexes:
            result.append(word_indexes[word])
    return result

In [38]:
def sentiment_encoder(sentiment: str):
    if sentiment == "positive":
        return 1
    return 0

#### Decoders

In [119]:
def review_decoder(encoded_review):
    encoded_review = list(filter(lambda x: x != 0, encoded_review))
    decoded_review = ""
    index_words = {index: word for word, index in word_indexes.items()}
    for encoded_word in encoded_review:
        decoded_word = index_words.get(encoded_word)
        if decoded_word:
            decoded_review += decoded_word + " "
    return decoded_review.strip()

### Encoding the data

In [39]:
train_data = train_data.apply(review_encoder)
test_data = test_data.apply(review_encoder)

In [40]:
train_data

33553    [124256, 65, 428, 14, 124257, 696, 8, 4, 166, ...
9427     [109, 698, 279, 1327, 8, 182, 17, 109, 282, 27...
199      [22, 944, 220, 8, 6, 664, 1279, 132, 19, 126, ...
12447    [61, 409, 11472, 7, 14, 127, 35, 1172, 296, 44...
39489    [73, 675, 302, 18, 71, 53, 124256, 417, 8, 66,...
                               ...                        
1559     [245, 7, 13258, 9, 196, 8, 97, 25, 1163, 29850...
13313    [86, 2953, 850, 7, 6, 5472, 452, 36, 634, 983,...
13528    [9, 6, 362, 335, 496, 9, 1055, 53, 7558, 7, 10...
25017    [125, 382, 12, 29, 554, 15, 124256, 387, 85, 1...
19317    [5, 17, 5, 5722, 4, 339, 7, 4, 207, 120, 47, 1...
Name: review, Length: 40000, dtype: object

In [41]:
test_data

7928     [718, 1393, 6, 10455, 77, 124256, 305, 9, 164,...
2107     [646, 8, 545, 43, 4105, 447, 39, 112, 29329, 1...
30363    [14, 16, 6, 81, 20, 34, 108, 81, 1844, 5, 658,...
31262    [6, 22, 444, 23, 6, 292, 67, 586, 30, 17, 1073...
42135    [16, 6, 5256, 7, 4, 6759, 3167, 53, 62, 4770, ...
                               ...                        
11284    [37449, 4, 1794, 5, 13304, 7, 4, 89, 20, 1246,...
44732    [124256, 258, 14, 20, 8, 29, 182, 735, 5, 994,...
38158    [14, 30, 9, 6, 384, 20, 51, 46, 12, 9, 55, 114...
860      [356, 16, 182, 6, 848, 18, 71, 124256, 422, 11...
15795    [9, 6, 543, 20, 123, 229, 338, 11, 58, 18, 71,...
Name: review, Length: 10000, dtype: object

In [42]:
train_labels = train_labels.apply(sentiment_encoder)
test_labels = test_labels.apply(sentiment_encoder)

In [43]:
train_labels

33553    1
9427     1
199      0
12447    1
39489    0
        ..
1559     0
13313    1
13528    0
25017    1
19317    1
Name: sentiment, Length: 40000, dtype: int64

In [44]:
test_labels

7928     0
2107     1
30363    1
31262    0
42135    1
        ..
11284    1
44732    1
38158    0
860      1
15795    1
Name: sentiment, Length: 10000, dtype: int64

### Applying padding to the data

In [45]:
max_len = 0
for td in train_data:
    len_td = len(td)
    if len_td > max_len:
        max_len = len_td

print("Training data max_len:", max_len)

max_len = 0
for td in test_data:
    len_td = len(td)
    if len_td > max_len:
        max_len = len_td

print("Test data max_len:", max_len)

Training data max_len: 1910
Test data max_len: 1473


In [46]:
train_data = tf.keras.preprocessing.sequence.pad_sequences(
    train_data, value=word_indexes["<PAD>"], padding="post", maxlen=2048)
test_data = tf.keras.preprocessing.sequence.pad_sequences(
    test_data, value=word_indexes["<PAD>"], padding="post", maxlen=2048)

### Building the model

In [47]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(124258, 16, input_length=2048),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [48]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [49]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 2048, 16)          1988128   
                                                                 
 global_average_pooling1d (G  (None, 16)               0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 16)                272       
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 1,988,417
Trainable params: 1,988,417
Non-trainable params: 0
_________________________________________________________________


### Training the model
I wasn't happy with the results so I trained one more time with increased batch_size

In [50]:
history = model.fit(train_data, train_labels, epochs=30, batch_size=512, validation_data=(test_data, test_labels))

Epoch 1/30
79/79 [==============================] - 23s 227ms/step - loss: 0.6930 - accuracy: 0.5049 - val_loss: 0.6928 - val_accuracy: 0.5538
Epoch 2/30
79/79 [==============================] - 19s 236ms/step - loss: 0.6924 - accuracy: 0.5871 - val_loss: 0.6917 - val_accuracy: 0.5318
Epoch 3/30
79/79 [==============================] - 17s 212ms/step - loss: 0.6903 - accuracy: 0.5791 - val_loss: 0.6883 - val_accuracy: 0.5352
Epoch 4/30
79/79 [==============================] - 13s 168ms/step - loss: 0.6831 - accuracy: 0.6718 - val_loss: 0.6771 - val_accuracy: 0.6404
Epoch 5/30
79/79 [==============================] - 14s 176ms/step - loss: 0.6656 - accuracy: 0.6980 - val_loss: 0.6538 - val_accuracy: 0.7323
Epoch 6/30
79/79 [==============================] - 12s 146ms/step - loss: 0.6352 - accuracy: 0.7480 - val_loss: 0.6202 - val_accuracy: 0.7416
Epoch 7/30
79/79 [==============================] - 11s 140ms/step - loss: 0.5976 - accuracy: 0.7615 - val_loss: 0.5824 - val_accuracy: 0.7792

In [52]:
history = model.fit(train_data, train_labels, epochs=30, batch_size=2048, validation_data=(test_data, test_labels))

Epoch 1/30
20/20 [==============================] - 5s 245ms/step - loss: 0.2228 - accuracy: 0.9196 - val_loss: 0.2941 - val_accuracy: 0.8854
Epoch 2/30
20/20 [==============================] - 6s 279ms/step - loss: 0.2212 - accuracy: 0.9200 - val_loss: 0.2936 - val_accuracy: 0.8856
Epoch 3/30
20/20 [==============================] - 5s 238ms/step - loss: 0.2199 - accuracy: 0.9207 - val_loss: 0.2935 - val_accuracy: 0.8858
Epoch 4/30
20/20 [==============================] - 6s 298ms/step - loss: 0.2188 - accuracy: 0.9212 - val_loss: 0.2926 - val_accuracy: 0.8862
Epoch 5/30
20/20 [==============================] - 6s 301ms/step - loss: 0.2176 - accuracy: 0.9215 - val_loss: 0.2929 - val_accuracy: 0.8851
Epoch 6/30
20/20 [==============================] - 7s 343ms/step - loss: 0.2164 - accuracy: 0.9226 - val_loss: 0.2921 - val_accuracy: 0.8868
Epoch 7/30
20/20 [==============================] - 6s 325ms/step - loss: 0.2149 - accuracy: 0.9229 - val_loss: 0.2914 - val_accuracy: 0.8867
Epoch 

In [53]:
loss, accuracy = model.evaluate(test_data, test_labels)

313/313 [==============================] - 1s 2ms/step - loss: 0.2839 - accuracy: 0.8891


### Testing the model

In [170]:
def get_random_review():
    index = np.random.randint(1, len(test_data))
    print("Random index:", index)
    test_sample = test_data[index]
    print("Review:", review_decoder(test_sample))
    return test_sample, test_labels.iloc[index]

In [197]:
def test_model():
    _review, _sentiment = get_random_review()
    random_review = np.array([_review])
    print("\nOriginal sentiment:", "positive" if _sentiment == 1 else "negative")
    prediction = model.predict(random_review)
    formatted_percentage = "{:.3f}%".format(prediction[0][0]*100)
    if prediction > 0.5:
        print("Model thinks it's a positive sentiment with the certainty of", formatted_percentage, "positivity")
    else:
        print("Model thinks it's a negative sentiment with the certainty of", formatted_percentage, "positivity")

In [198]:
test_model()

Random index: 1321
Review: reading the comment made about this movie and currently watching it I can understand how the person felt about it decisions made were after listening to common sense the movie came out I had heard the information as to how it came about storyline was made from an actual event an award show an actor thanking the li'l people attributed the award to a former school teacher unexpectedly outing the course many people come 'out' of the closet most every day outing is different for each person real life the outcome of any individual is gonna be different as well a willingness to accept who they are is the most important thing in life to reach personal happiness those around them the joy and honest acceptance can make life much more fuller the movie the outcome of how is out'ed is a lot more comical than real life the acceptance of the community showed the others that was himself and nothing the performances were crazy memorable quotes and use of music add to the out

In [199]:
test_model()

Random index: 8098
Review: directors crossing over to to make movies is nothing new with the temporary exodus of the likes of in the 90s their collective output only a few movies may I say just made an impact at the box office and partnership has been a tour de force in recent cinematic history especially with their now famous trilogy which was remade into so it's no surprise when comes knocking on the without fellow collaborator who usually has duties how did fare with writers and like the without the in the way the story is crafted the characters and the parallels drawn with the series and pair up ala and only that they don't belong to any federal investigative agency who bear arms but are employees of who's chief role is to ensure that sexual predators who belong to their jurisdiction are kept safe from society when they are released from having served time they are the shepherds tending to their flock only that their flock suffer from sick sexual perversion with the propensity for 

In [200]:
test_model()

Random index: 2872
Review: 2054 a computer generated system controls the city and when a young woman is kidnapped detective must go against to find is a splendid blend of film making mixed with a conceptual futuristic narrative that lights up the screen in a shocking manor with a noir themed ideology and conceptual montages that should delight are the animation masters numerous winning films are endless from the charming to the mystifying and so any company or director has a real challenge to knock them of their perch isn't a film aimed for the young audience though and like 2007's brings a strong and mature approach to the genre of animation to make an older and more challenging film to its targeted older 2005 released a shockingly brilliant noir that shook up the whole usage of green screen with a splendid balance of filming in black and white with the odd spurts of colour and a year later took up a similar approach with this equally visually masterful stroke of film picture however 

In [201]:
test_model()

Random index: 2535
Review: I rented recently mainly because I read that it was this film that inspired to make however is more of a thriller than a drama film is quite political but I agree strongly with its message nuclear power though extremely efficient is far too dangerous for common use risks are simply too high the persons in charge of real nuclear plants are far more responsible and ethical than those depicted in the film given that the at happened mere months after this film was released puts that in question fairness that case was probably more one of incompetence than of is not just critical of nuclear power but also of modern news media similar in vein to only much more serious a print journalist myself I am quite familiar with how people perceive the media but it was a little frightening to see that even in the '70's news was already selling out character tries to convince her boss to let her do real news instead of fluff she is advised not to try the change as finds that p

In [202]:
test_model()

Random index: 6882
Review: I had my doubts about another love story wherein disabled individuals find meaning and redemption through honest communication it's still not at the top of my list the performances from and and exemplary almost stunning and rescue this from being just another depiction of an victim is strong perhaps even overdone at times her dialog dissolves into undistinguishable the overall effect is commendable and rewarding may be the perfect compliment to her

Original sentiment: positive
1/1 [==============================] - 0s 32ms/step
Model thinks it's a positive sentiment with the certainty of 97.275% positivity


In [203]:
test_model()

Random index: 7019
Review: I tried I really did I thought that maybe if I gave another chance I could enjoy his movie I know that after seeing I felt ill and nearly disgusted to the core but some of the reviews were quite good and in favor so I was like the hell least you didn't pay 10 dollars at the it a it's better to go to your dentist and ask for a root canal without any previous anesthetic to alleviate the horror of so much pain I often wonder if it wouldn't be better to go back to my childhood and demand my former bullies to really let me have it other occasions I often think that the world is really flat and that if I sail away far enough I will not only get away from it all but fall clear over and that some evil thing will snatch me with its 9000 tentacles and squeeze the life and some french fries from 1995 still lingering inside my esophagus out of there a reason for say not at just that maybe her thought that writing a story centered on her madness that makes look like look 

In [204]:
test_model()

Random index: 8959
Review: movie is I don't even know where to begin speechless I can't even describe how awful this is blood is flourescent first of all and the acting is only good part was the biker chick that saves the day movie was rediculous I don't see how it could even get a vote of 1 its so bad looks like it was made by highschool students

Original sentiment: negative
1/1 [==============================] - 0s 27ms/step
Model thinks it's a negative sentiment with the certainty of 3.107% positivity


In [205]:
test_model()

Random index: 9678
Review: I normally don't like romantic films but I love this film very much story is really touching and the ending is very appropriate I feel I really care for many characters in this film I feel I can feel their feelings most romantic films always make me feel detached and bored this one completely makes me feel involved starting from the scene of running along the beach with until the ending scene I want to rate this film 11 out of 10 because I want to give an extra one point for the ending acting part is very strong has a good opportunity to show her talent not impressed with in on this film completely changes my viewpoint towards him so irresistibly charming here is as terrific as film might not be as good complex deep or believable as other films which deal a little bit with the same kind of relationship such as de la chair' or coitum animal but can still be proud of itself as it casts a rare different light on that kind of relationship that difference makes th

In [206]:
test_model()

Random index: 8734
Review: this as a young naive punk when it was first released me snifflin' like a baby as I left the theatre trying not to let anyone see when I saw it again now in '07 I knew what to expect the sobs were ready primed as their required moment approached this time I was at I hadn't remembered from my youthful or perhaps hadn't noticed because of it was the technical brilliance of this movie use of flashbacks which tell so much story without resorting to dialogue camera work which seemed to place the viewer together with the characters in the scene of the opening when is crossing the street to the diner the camera pans behind the woman child sitting on a bench in the foreground framing the street scene story itself the characters seedy sad brutally real is very touching to be drawn so closely into a human drama such as this with people most of us would likely spurn again could be any of us have been '70 when I saw it I recall that upon leaving the theatre I was impelle

In [207]:
test_model()

Random index: 1153
Review: is a great story and was just the beginning of equality in the are still working on it despite the fact this is true it's still a movie and this is a movie site I realize independent films have a hard time getting good actors but wow only one even mediocre is the excellent even he couldn't make up for all the actors the one playing him as a young absolutely atrocious acting the script was terribly cliché but even then you have got to get some decent actors I wouldn't recommend this to anybody because it is so poorly done in every category some books about the true story of the because they give these men the respect they deserve

Original sentiment: negative
1/1 [==============================] - 0s 132ms/step
Model thinks it's a negative sentiment with the certainty of 4.953% positivity


In [208]:
test_model()

Random index: 8198
Review: 1 let's suspend belief for a moment and let's stop pretending we could might or ought know it or to there in space knowledge in that area is probably primitive as say middle ages maps are compared to today's satellite maps so we really have no clue 2 considering this is a it gets much better than many big budget blockbusters and that is just incredible 3 all in all a show worth watching as it portrays the enhanced and fictionalized account of what we know of the solar system this far 4 probably fictionalizing and the whole thing is the only way to make it palatable to a large public watched clips from space missions and space probes quality is generally average to poor and the comparison would be between looking at a chest what it tells about the human body and compare it with a cyborg one would be most entertaining the chest is real while the cyborg flick is just fictionalized 5 actors do a good job i'll tell my grandchildren about but very fair for it being

In [209]:
test_model()

Random index: 8125
Review: and time again it seems that the comedic actors of are surprising me with their talents as dramatic performers first it was then it was being one then than and now that's absolutely right the guy who has based an entire career on making brainless goofball comedies not complaining always been a has finally given me a performance of which he can be proud course some readers may be wondering whatever happened to but I can only assure you that it's a film currently taking measures to see as soon as possible since heard it's terrific first film from director is an intelligent and respectful exploration of grief and friendship it does occasionally stray into territory the somewhat predictable storyline proves surprisingly deep and moving with both and giving memorable is a successful dentist with a good family though he finds himself unable to communicate adequately with his wife and so his marriage is a little shaky is roommate a man who lost his entire family in 

In [210]:
test_model()

Random index: 1473
Review: is a bit of a first for me the first time I have ever been disappointed in a film isn't a bad film sets costumes and the odd great but it could have been made by any hollywood director pacing was very odd the last third was just spent waiting for the film to end by myself and the cast was excellent probably the only pleasure in the film back

Original sentiment: negative
1/1 [==============================] - 0s 20ms/step
Model thinks it's a negative sentiment with the certainty of 42.365% positivity


In [211]:
test_model()

Random index: 4908
Review: people are making fun out of acting abilities I really see why if I remember his other films are acceptable to his reading reviews about I didn't expect anything from this film I barely came through and I read a lot that was even worse I was really surprised about this one my opinion it is one of his very best movies it had have more action sequences and a couple more fighting scenes it would be best I liked the story the directing and the actors was just incredible will have a great in the future few action scenes were very well made was very believable in his unusual role

Original sentiment: positive
1/1 [==============================] - 0s 27ms/step
Model thinks it's a positive sentiment with the certainty of 96.636% positivity


In [212]:
test_model()

Random index: 2810
Review: to start I was only able to stomach about the first 10 minutes before I turned it off in disgust from the actor playing himself the rest were just terrible I can only stretch my suspension of disbelief only so the very opening of the first episode I lost count of how many errors plot holes and horrible costumes there were began with some poor peasant trying to hunt for a deer to feed his family well and good the poor blighter must have been mostly deaf because a handful of soldiers in full armour on horseback were able to sneak up on him to within about 10 as he's running away he goes from having them 10 feet behind him to a shot where you cannot even see them at all immediately followed by them about 20 feet behind him again he runs into some bushes and is immediately manhandled by two of the just mere seconds before were galloping on horseback dozens of feet behind on the soldiers is so painfully obviously cloth which they tried to make look like maille and

In [213]:
test_model()

Random index: 1438
Review: the eagerly awaits the launch of series two of series one just finished showing did not group of over forty passengers struggle to cope after their plane into a deserted island pray for rescue as the days turn into weeks and the survivors explore the rainforest that surrounds them they begin to wonder whether they are the series has a hard time keeping up the pace after the explosive pilot episodes which hurls at the group a polar bear a giant beastie and the possibility of others on the island the series manages to pick itself up after a few episodes and is outstanding by the final episode the greatest thing about is the fact that we visit each character's life before the crash in a series of flashbacks gives us more insight and hence more suspense and excitement as the events unfold in the this show is a superb combination of drama suspense and the supernatural it is quite simply 1 the cliffhanger that was the end of 1 of 2 begins with our protagonists delv

In [214]:
test_model()

Random index: 5383
Review: got to say a big fan of these on the even the ones that most people seem to hate are often held in relatively high esteem by me but one of these sorts of films that I didn't like much was and unfortunately it would seem that trains and on the don't mix as is another lacklustre that this sort of film really needs is a resoundingly nasty lead character and while offers up three potential candidates not one of them steps up and becomes this villain leaving the lacking in the most important area actually gets off to a good start as three young men on a passenger train begin slightly irritating the guests on board leads the audience to believe that there is more in store but unfortunately it never really gets going once the scene has been set there the trio end up 'taking over' the train and use their new found power to terrorise the guests and rape the has to be said that there's a fair amount of sleaze in this film which will be pleasing to many viewers but ther

In [215]:
test_model()

Random index: 1964
Review: is a movie and nothing more for that it is great fun to watch it skims over political issues so what I am sure she wasn't trying to make and here not try to make it anything else but what it fare very enjoyable at we remember what 1984 was like become very sophisticated according to the media as far as what we watch or not I tend to differ on this point knew this was and she went ahead and did her lightest fare and the remake of knows what she is plays every role for the camp that you can get out of it just knows herself really well and she knows what she can do really has always made me laugh cheerfully and innocently I loved her in and every thing she's ever been in has never tried to be anything else but who she that's that bubbling giggly girl next door who happens to be very pretty and has a smile and a laugh that will always endear me and remind me that life is pretty short and you've just got to lighten up because before you know are old wrinkled and s

In [216]:
test_model()

Random index: 5180
Review: channel surfing we found this movie with its promising synopsis were dismayed at the flat acting and formulaic storyline found amusing the exploding car scenes unbelievable shoot outs and sets that crash down with the weight of and cardboard was even more hilarious was seeing all the recycled scenes from where the church front falls on the school bus the store fronts break away from the main street the overpass collapsing and the red truck speeding down the alley way as the bricks fall down on it was a good laugh although unintended and leaves the viewer wondering if it was not originally intended as a low budget spoof of disaster films

Original sentiment: negative
1/1 [==============================] - 0s 22ms/step
Model thinks it's a negative sentiment with the certainty of 3.744% positivity


In [217]:
test_model()

Random index: 3447
Review: I have to admit right off the top not a big fan of films these days of them are sentimental crap this one like the previous film from is a lot of fun two lead characters were perhaps a bit too compared to the two leads in but otherwise this film is but the rest of the film more than made up for it animation looked great the humor though broad was consistently especially liked line I hadn't promised on her deathbed that I wouldn't kill you I would kill and the actors doing the voices except the two leads were all doing an animated movie what a like everyone else I loved the outtakes I hope the video has the new ones

Original sentiment: positive
1/1 [==============================] - 0s 21ms/step
Model thinks it's a positive sentiment with the certainty of 77.558% positivity


In [218]:
test_model()

Random index: 7972
Review: I saw recently saw this at the 2007 film and I can't believe that this is official submission to the 79th to be considered for best foreign film last year also had official submission for best foreign film with I wasn't personally a fan of but at least it had some production value to it could have been shot as a high school film project a short film it may have worked but as a feature film it is as lost as it's characters is a story about five twenty something friends who live in a small town and have no goals or hopes or ambitions for the future and like to live in the past of their long gone childhood isn't like some remote northern village a popular summer resort with a population of city and townships of almost and lies between two major southern coastal cities of and characters are likable enough guys but if they were high school age kids with a bleak or confused attitude about what they plan to do with their lives I would undoubtedly care about them mor

In [219]:
test_model()

Random index: 9332
Review: is a typically entertaining movie of the early 1930s you have in the lead these films are even better just fun stuff to watch when films are it's a negative but not so with films from with the slang and the attitudes dress hairstyles they are dated but that's a big part of the fun films have an edge to them that almost always are fun to also have a corniness which is appealing and fascinating see people like the juvenile delinquents pictured in this film and their goofy parents that you just don't see in any period but this one on this movie the kids go before the judge and you sit and just laugh at these crazy characters that appear in court on behalf of their kids one after the other we get the stereotypical emotional father the dad the mom and a few other moms who all in dramatic form plead theirs is good though things are predictable in some cases you don't mind because everyone in here is so much fun to also teaches you that kids were punks 75 years ago 

In [220]:
test_model()

Random index: 5214
Review: I watched of this movie and wondered why it got such horrible reviews here was fairly easy to watch had good casting role of the sociopath serial killer was very believable he was both charasmatic and chilling rest of the main characters weren't so bad is your typical movie married couple cannot conceive so they go to a fertility clinic for help sociopathic donor fixates on the recipients and in typical stalker form intrudes into their I said most of the movie was fairly we see grow more and more obsessed in raising his baby and creating the perfect family with the mother course things don't work out for him the way he planned a bad plot the last 15 minutes were just horrible I am pretty tolerant with movies at but I was just amazed at how bad the ending was written I actually scoffed in all not the worst movie seen but I wouldn't be able to sit through it again I skipped the only redeaming quality here was role one of the best serial killers looking around a

In [221]:
test_model()

Random index: 8190
Review: ever ever ever do anything right movie was meant to be seen on in a letterbox format doesn't even believe in pan and scan we see whole scenes where a shoulder on the left side of the screen talks to a shoulder on the right side course not that you are missing much movie is incredibly bad very hard to enjoy a film where characters are screaming at the top of their lungs during of the movie for no reason

Original sentiment: negative
1/1 [==============================] - 0s 20ms/step
Model thinks it's a negative sentiment with the certainty of 17.375% positivity
